___
**📌 Baseline From The Notebooks Below**

>  **First NoteBook:** [DeBERTa-v3-SMALL Starter - CV 0.820 LB 0.800](https://www.kaggle.com/code/cdeotte/deberta-v3-small-starter-cv-0-820-lb-0-800) 🙏

>  **Second NoteBook:** [Huge Ensemble](https://www.kaggle.com/code/thedevastator/huge-ensemble) 🙏


### Custom Model Structure

![](https://cdn.sanity.io/images/vr8gru94/production/2425dc0efd3f73a0bf57b3bf85a091c78619ec2c-1920x1110.png)

- **Adding Mean Pooling on last hidden states**
=> I've replaced the `ContextPooler` with a `mean pooling layer`

- **Adding BiLSTM Layers** (Optional) - For me it was worsen CV scores
- **Automatic Mixed Precision = False** - FP16 in some cases hurts the performance of transformers.

- **Loss Function: MSE -> RMSE** (Optional)

<div style="border-radius:10px; border: #FFA500 solid; padding: 15px; font-size:100%;">

📌 **Improvement Ideas**

- **Increase Pos Embeddings Size** - max_position_embeddings(512 -> 1024)
    
- **Utilizing Different Layers** - There are many ways to do this concat, mean pooling, max pooling, attention, lstm, hierarchical or parallel aggregation.
   
- **Unsupervised Data Augmentation** ex) persuade corpus

- **LLRD** - LayerWise Learning Rate Decay(*Experiment: LLRD vs General Learning Rate Decay*)

- **Ensembling Techniques** - Hill Climbing

___

# <div style="padding: 25px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFA500"><b><span style='color:#FFA500'></span></b> <b>1. Initial Setting</b></div>

In [ ]:
import os 
os.environ['CUDA_IS_VISIBLE'] = '0,1'
import sys
import gc
import re
import ctypes
import random
from tqdm import tqdm
import polars as pl

import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.checkpoint import checkpoint

from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, AutoConfig
from transformers.modeling_utils import PretrainedConfig, PreTrainedModel

from transformers import Trainer, TrainingArguments
from datasets import Dataset
import warnings
warnings.filterwarnings('ignore')

In [ ]:
class CFG:
    SEED = 2024
    VER = 1
    INFERENCE = True
    gradient_checkpointing = False
    FREEZING_EMBEDDINGS = True
    FREEZE_LAYERS = 1 # Only freezing embedding
    BASE_PATH = '/kaggle/input/learning-agency-lab-automated-essay-scoring-2/'
    preset = '/kaggle/input/aes2-deberta-pooling'
    MAX_LEN = 1024
    TRAIN_BATCH = 4
    EVAL_BATCH = 8
    EPOCHS = 1

In [ ]:
Clean = True

def clean_memory():
    if Clean:
        ctypes.CDLL('libc.so.6').malloc_trim(0)
        gc.collect()
        
clean_memory()    

In [ ]:
def seed_everything():
    random.seed(CFG.SEED)
    os.environ['PYTHONHASHSEED'] = str(CFG.SEED)
    np.random.seed(CFG.SEED)
    torch.manual_seed(CFG.SEED)
    torch.cuda.manual_seed(CFG.SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything()

# <div style="padding: 25px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFA500"><b><span style='color:#FFA500'></span></b> <b>2. Road and Read Data</b></div>

In [ ]:
df_train = pd.read_csv(CFG.BASE_PATH + 'train.csv')
df_train = df_train.sort_values(by='essay_id')
df_train['label'] = df_train['score'] - 1

skf = StratifiedKFold(n_splits=5, random_state=CFG.SEED, shuffle=True)
for i, (_, val_index) in enumerate(skf.split(df_train,df_train['score'])):
    df_train.loc[val_index, 'fold'] = i

print('Shape of Train: ', df_train.shape)
print(display(df_train.head()))

In [ ]:
df_test = pd.read_csv(CFG.BASE_PATH + 'test.csv')
df_test = df_test.sort_values(by='essay_id')
print('Shape of Test: ', df_test.shape)
print(display(df_test.head()))

In [ ]:
cList = {
  "ain't": "am not","aren't": "are not","can't": "cannot","can't've": "cannot have","'cause": "because",  "could've": "could have","couldn't": "could not","couldn't've": "could not have","didn't": "did not","doesn't": "does not","don't": "do not","hadn't": "had not","hadn't've": "had not have","hasn't": "has not",
  "haven't": "have not","he'd": "he would","he'd've": "he would have","he'll": "he will","he'll've": "he will have","he's": "he is",
  "how'd": "how did","how'd'y": "how do you","how'll": "how will","how's": "how is","I'd": "I would","I'd've": "I would have","I'll": "I will","I'll've": "I will have","I'm": "I am","I've": "I have",
  "isn't": "is not","it'd": "it had","it'd've": "it would have","it'll": "it will", "it'll've": "it will have","it's": "it is","let's": "let us","ma'am": "madam","mayn't": "may not",
  "might've": "might have","mightn't": "might not","mightn't've": "might not have","must've": "must have","mustn't": "must not","mustn't've": "must not have","needn't": "need not","needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not","oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
  "shan't've": "shall not have","she'd": "she would","she'd've": "she would have","she'll": "she will","she'll've": "she will have","she's": "she is",
  "should've": "should have","shouldn't": "should not","shouldn't've": "should not have","so've": "so have","so's": "so is","that'd": "that would","that'd've": "that would have","that's": "that is","there'd": "there had","there'd've": "there would have","there's": "there is","they'd": "they would","they'd've": "they would have","they'll": "they will","they'll've": "they will have","they're": "they are","they've": "they have","to've": "to have","wasn't": "was not","we'd": "we had",
  "we'd've": "we would have","we'll": "we will","we'll've": "we will have","we're": "we are","we've": "we have",
  "weren't": "were not","what'll": "what will","what'll've": "what will have",
  "what're": "what are","what's": "what is","what've": "what have","when's": "when is","when've": "when have",
  "where'd": "where did","where's": "where is","where've": "where have","who'll": "who will","who'll've": "who will have","who's": "who is","who've": "who have","why's": "why is",
  "why've": "why have","will've": "will have","won't": "will not","won't've": "will not have","would've": "would have","wouldn't": "would not",
  "wouldn't've": "would not have","y'all": "you all","y'alls": "you alls","y'all'd": "you all would",
  "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you had","you'd've": "you would have","you'll": "you you will","you'll've": "you you will have","you're": "you are",  "you've": "you have"
   }

In [ ]:
c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

def removeHTML(x):
    html = re.compile(r'<.*?>')
    return html.sub(r'',x) # html -> ''

def dataPreprocessing(x):
    
    x = x.lower()
    # remove html
    x = removeHTML(x)
    
    #remove string with starting @
    x = re.sub("@\w+",'',x)
    
    # remove number
    x = re.sub("'\d+",'',x)
    x = re.sub("\d+",'',x)
    
    # remove url
    x = re.sub("http\w+",'',x)
    
    # Replace consecutive empty spaces with a single empty spaces
    x = re.sub(r'\s+', " ", x)
    # Replace consecutive commas and periods with a single comma and period
    x = expandContractions(x)
    x = re.sub(r'\.+', ".",x)
    x = re.sub(r'\,+', ",",x)
    x = re.sub(r'[^\w\s.,;:""''?!]', '', x)
    # Remove empty character at the beginning and end
    x = x.strip()
    return x

In [ ]:
train = pl.from_pandas(df_train)
test = pl.from_pandas(df_test)

train = train.with_columns(
         pl.col('full_text').map_elements(lambda x: dataPreprocessing(x))
)
df_train = train.to_pandas()

test = test.with_columns(
         pl.col('full_text').map_elements(lambda x: dataPreprocessing(x))
)
df_test = test.to_pandas()
df_train['label'] = df_train['label'].astype('float32')

In [ ]:
training_args = TrainingArguments(
    output_dir = f'/output_v{CFG.VER}',
    per_device_train_batch_size=CFG.TRAIN_BATCH,
    per_device_eval_batch_size=CFG.EVAL_BATCH,
    num_train_epochs= CFG.EPOCHS,
    evaluation_strategy='steps',
    save_strategy='steps',
    logging_steps=125,
    save_steps=250,
    load_best_model_at_end=True,
    weight_decay=1e-2, 
    fp16=False,# automatic mixed precision, AMP
    learning_rate=2e-5, # 1e-5 -> 2e-5
    lr_scheduler_type='linear',
    optim='adamw_torch',
    metric_for_best_model='qwk',
    save_total_limit=1,
    report_to='none',
    gradient_checkpointing=CFG.gradient_checkpointing,
    gradient_accumulation_steps=2,
)

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    qwk = cohen_kappa_score(labels, predictions.clip(0,5).round(0), weights='quadratic')
    results = {
        'qwk': qwk
    }
    return results

# <div style="padding: 25px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFA500"><b><span style='color:#FFA500'></span></b> <b>3. Pooling Introduction</b></div>

**MeanPooling**

In [ ]:
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9) #
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings  # 1 X 768

**MeanMaxPooling**

![](https://www.googleapis.com/download/storage/v1/b/kaggle-forum-message-attachments/o/inbox%2F16438831%2F952609a0cdb574a3621c438d960e1471%2Fmax%20pooling.JPG?generation=1714549387950952&alt=media)

In [ ]:
class MeanMaxPooling(nn.Module):
    def __init__(self):
        super(MeanMaxPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        mean_pooling_embeddings = torch.mean(last_hidden_state, 1)
        _, max_pooling_embeddings = torch.max(last_hidden_state, 1)
        mean_max_embeddings = torch.cat((mean_pooling_embeddings, max_pooling_embeddings), 1)
        return mean_max_embeddings # 1 X 1536

**Attention Pooling** (https://www.kaggle.com/competitions/feedback-prize-english-language-learning/discussion/361678)

In [ ]:
class AttentionPooling(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.attention = nn.Sequential(
           nn.Linear(in_dim, in_dim),
           nn.LayerNorm(in_dim),
           nn.GELU(),
           nn.Linear(in_dim, 1),
        )
    def forward(self, last_hidden_state, attention_mask):
        w = self.attention(last_hidden_state).float()
        w[attention_mask==0]=float(
        '-inf')
        w = torch.softmax(w,1)
        attention_embeddings = torch.sum(w * last_hidden_state, dim=1)
        return attention_embeddings
    

# <div style="padding: 25px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFA500"><b><span style='color:#FFA500'></span></b> <b>4. Customizing Model</b></div>

In [ ]:
if CFG.INFERENCE is None:
    config = AutoConfig.from_pretrained(CFG.preset)
    config._name_or_path = CFG.preset
    config.num_labels = 1
    config.attention_probs_dropout_prob = 0.0 # For Regression
    config.hidden_dropout_prob = 0.0 # For Regression
    config.max_position_embeddings = 1024 # 512 -> 1024
    # config.save_pretrained('./config.json')
    base_model = AutoModelForSequenceClassification.from_pretrained(CFG.preset, config=config)

In [ ]:
class Custom_Model(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.base_model = model
        
        if CFG.FREEZING_EMBEDDINGS:
            print('Freezing embeddings.')
        for param in self.base_model.deberta.embeddings.parameters():
            param.requires_grad = False
        
        if CFG.FREEZE_LAYERS > 0:
            print(f'Freezing {CFG.FREEZE_LAYERS} layers')
            for layer in self.base_model.deberta.encoder.layer[:CFG.FREEZE_LAYERS]:
                for param in layer.parameters():
                    param.requires_grad = False
          
        self.pooler1 = MeanPooling() # Optional 
        self.pooler2 = MeanMaxPooling() # Optional
        self.pooler3 = AttentionPooling(self.base_model.config.hidden_size)
        # self.LSTM = nn.LSTM(input_size=self.base_model.config.hidden_size, hidden_size=self.base_model.config.hidden_size // 2,
        #                     num_layers=2, batch_first=True,dropout=self.base_model.config.hidden_dropout_prob, bidirectional=True)
        self.fc1 = nn.Linear(in_features = self.base_model.config.hidden_size, out_features = 1)
        self.fc2 = nn.Linear(in_features = self.base_model.config.hidden_size*2, out_features = 1)
        
    def forward(self, input_ids, attention_mask, labels=None):
        output = self.base_model(input_ids=input_ids,
                         attention_mask=attention_mask,
                         labels = labels,
                         output_hidden_states = True)
        
        #Mean Pooling
        # pooled_output1 = self.pooler1(output.hidden_states[-1], attention_mask)
        # logits = self.fc1(pooled_output1) 
        
        # Mean-max Pooling
        # pooled_output2 = self.pooler2(output.hidden_states[-1], attention_mask)
        # logits = self.fc2(pooled_output2) 
        
        # Attention Pooling
        pooled_output3 = self.pooler3(output.hidden_states[-1], attention_mask)
        logits = self.fc1(pooled_output3)
        
        # You can change MSE Loss -> RMSE Loss
        if labels is not None:
            loss_fct = nn.MSELoss()
            mse_loss = loss_fct(logits.squeeze(), labels.squeeze())
            rmse_loss = torch.sqrt(mse_loss)
            return mse_loss, logits
        else:
            return logits

class Custom_Model(PreTrainedModel):
    def __init__(self, config, model):
        super().__init__(config)
        self.base_model = model
          
        self.pooler1 = MeanPooling() # Optional 
        self.pooler2 = MeanMaxPooling() # Optional
        # self.LSTM = nn.LSTM(input_size=self.base_model.config.hidden_size, hidden_size=self.base_model.config.hidden_size // 2,
        #                     num_layers=2, batch_first=True,dropout=self.base_model.config.hidden_dropout_prob, bidirectional=True)
        self.fc1 = nn.Sequential(
            nn.Linear(config.hidden_size, config.hidden_size),
            nn.ReLU(),
            nn.Linear(in_features = config.hidden_size, out_features = 1)
        )
        
        self.fc2 = nn.Sequential(
            nn.Linear(config.hidden_size*2, config.hidden_size*2),
            nn.ReLU(),
            nn.Linear(in_features = config.hidden_size*2, out_features = 1)
        )
    def forward(self, input_ids, attention_mask, labels=None):
        output = self.base_model(input_ids=input_ids,
                         attention_mask=attention_mask,
                         labels = labels,
                         output_hidden_states = True)
        
        #Mean Pooling
        pooled_output1 = self.pooler1(output.hidden_states[-1], attention_mask)
        logits = self.fc1(pooled_output1) 
        
        # Mean-max Pooling
        # pooled_output2 = self.pooler2(output.hidden_states[-1], attention_mask)
        # logits = self.fc2(pooled_output2) 
    
        # You can change MSE Loss -> RMSE Loss
        if labels is not None:
            loss_fct = nn.MSELoss()
            mse_loss = loss_fct(logits.squeeze(), labels.squeeze())
            rmse_loss = torch.sqrt(mse_loss)
            return mse_loss, logits
        else:
            return logits

# <div style="padding: 25px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFA500"><b><span style='color:#FFA500'></span></b> <b>5. Training Model</b></div>

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(CFG.preset,  truncation=True, max_length=CFG.MAX_LEN, clean_up_tokenization_spaces=False)
def preprocess(sample):
    return tokenizer(sample['full_text'],)

In [ ]:
if CFG.INFERENCE is None:
    for i in range(1):
        train = df_train[df_train['fold'] != i]
        valid = df_train[df_train['fold'] == i]

        
        dataset_v = Dataset.from_pandas(valid)
        dataset_t = Dataset.from_pandas(train)
        dataset = Dataset.from_pandas(df_train)
        # Valid Dataset
        tokenized_dataset_v = dataset_v.map(preprocess, num_proc=2, 
                         remove_columns=['essay_id', 'full_text','score'])
        # Train Dataset
        tokenized_dataset_t = dataset_t.map(preprocess, num_proc=2, 
                        remove_columns=['essay_id', 'full_text','score'])
        
        
        trainer = Trainer(
                 model = Custom_Model(base_model),
                 args = training_args,
                 tokenizer = tokenizer,
                 train_dataset = tokenized_dataset_t,
                 eval_dataset = tokenized_dataset_v,
                 data_collator = DataCollatorWithPadding(tokenizer, padding='longest'),
                 compute_metrics=compute_metrics
                 )

        trainer.train()
        
        y_true = valid['score'].values
        predictions = trainer.predict(tokenized_dataset_v).predictions
        predictions = predictions + 1
        cm = confusion_matrix(y_true, predictions.clip(1,6).round(), labels=[x for x in range(1,7)])
        draw_cm = ConfusionMatrixDisplay(confusion_matrix=cm,
                                  display_labels=[x for x in range(1,7)])
        draw_cm.plot()
        plt.show()
        
        trainer.model.config = config
        config.save_pretrained(f'DeBerta_Small_v{CFG.VER}_Fold{i}')
        trainer.save_model(f'DeBerta_Small_v{CFG.VER}_Fold{i}')

        del train, valid, tokenized_dataset_t, tokenized_dataset_v, trainer
        clean_memory()
        
else:
    None

# <div style="padding: 25px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFA500"><b><span style='color:#FFA500'></span></b> <b>6. Inference</b></div>

In [ ]:
if CFG.INFERENCE:
    dataset_test = Dataset.from_pandas(df_test)
    tokenized_test = dataset_test.map(preprocess, num_proc=3, 
                         remove_columns=['essay_id', 'full_text'])
    model = AutoModelForSequenceClassification.from_pretrained(CFG.preset)
    
    trainer = Trainer(
                 model = model,
                 args = training_args,
                 tokenizer = tokenizer,
                 data_collator = DataCollatorWithPadding(tokenizer, padding='longest'),
                 compute_metrics=compute_metrics
                 )
    preds_test = trainer.predict(tokenized_test).predictions
    preds_test = preds_test + 1
    
    sub = pd.DataFrame({'essay_id': df_test.essay_id.values})
    sub['score'] = preds_test.clip(1,6).round()
    sub.to_csv('submission.csv',index=False)
    print('Submission shape', sub.shape)
    print(sub.head())